In [1]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from Parser import parse_frames
from scipy.signal import medfilt
from Functions import analyse_each_rep, find_extremas
from JointAngles import JointAngles
import numpy as np
import glob
import os
import pickle
import pandas as pd
from functools import reduce

In [61]:
input_folder = "C:\\Users\\altaok\\Documents\\GitHub\\IndividualProject\\keypoints_for_all"
folder_paths = glob.glob(os.path.join(input_folder, 'output_points_bicep*'))


def fill_dataframe(keypoints_paths):
    angle_label_dict = {}
    data = {}
    labels = []
    df_list = []
    for folder in folder_paths:
        video_name = os.path.basename(folder).replace('output_points_', '')
        label = 0 if 'correct' in folder else 1
        frame_poses = parse_frames(folder)
        joint_angles = JointAngles('bicep curl', frame_poses)

        upArm_trunk_angles = np.array(joint_angles.upArm_trunk_angles, dtype=np.float)
        upArm_trunk_angles = np.nan_to_num(upArm_trunk_angles)
        upArm_trunk_angles_filtered = medfilt(medfilt(upArm_trunk_angles, 5), 5)

        
        upArm_forearm_angles = np.array(joint_angles.upArm_forearm_angles, dtype=np.float)
        upArm_forearm_angles = np.nan_to_num(upArm_forearm_angles)
        upArm_forearm_angles_filtered = medfilt(medfilt(upArm_forearm_angles, 5), 5)
       

        trunk_knee_angles = np.array(joint_angles.trunk_knee_angles, dtype=np.float)
        trunk_knee_angles = np.nan_to_num(trunk_knee_angles)
        trunk_knee_angles_filtered = medfilt(medfilt(trunk_knee_angles, 5), 5)
        
        #number of reps 
        upArm_forearm_maximas, indexesmax3 = find_extremas(upArm_forearm_angles_filtered) 
       
        if upArm_forearm_maximas.size == 0:
            s1 = pd.Series([upArm_forearm_angles_filtered, upArm_trunk_angles_filtered,  trunk_knee_angles_filtered], name='Angle_array')
            s2 = pd.Series([label, label, label], name='Label')
            df = pd.concat([s1,s2], axis=1)           
            df_list.append(df)
           
            
        else:         
            # Extract rep angles
            each_rep_angles = analyse_each_rep('dataset', trunk_knee_angles_filtered, upArm_forearm_angles_filtered, upArm_trunk_angles_filtered, upArm_forearm_maximas)            
            for n in each_rep_angles:
                labels.append(label)    
                               
            s1 = pd.Series(each_rep_angles, name='Angle_array')
            s2 = pd.Series(labels, name='Label')
            df = pd.concat([s1,s2], axis=1)
        
            df_list.append(df)
            # Erase labels
            labels = []
            
    
    if len(df_list) > 1:
        df_final = pd.concat(df_list, axis=0).reset_index(drop=True)
       
    return df_final


        

In [62]:
df = fill_dataframe(folder_paths)
print(df.info())
print(df)

    

Repetition: 1
Minimum angle between upper arm and forearm: 79.2930066508191
Maximum angle between upper arm and trunk: 15.940260500073242
Maximum angle between trunk and knee: 170.820294598774


Repetition: 2
Minimum angle between upper arm and forearm: 53.04660650807649
Maximum angle between upper arm and trunk: 14.970649131653595
Maximum angle between trunk and knee: 178.23144888017174


Repetition: 3
Minimum angle between upper arm and forearm: 50.73602659413513
Maximum angle between upper arm and trunk: 14.589350087609121
Maximum angle between trunk and knee: 178.650769462795


Repetition: 4
Minimum angle between upper arm and forearm: 49.87618603557552
Maximum angle between upper arm and trunk: 15.442658775413246
Maximum angle between trunk and knee: 177.744060311366


Repetition: 5
Minimum angle between upper arm and forearm: 50.30771548496408
Maximum angle between upper arm and trunk: 15.415886780389865
Maximum angle between trunk and knee: 178.2383216423263
Repetition: 1
Minimu

Repetition: 1
Minimum angle between upper arm and forearm: 52.28476162665575
Maximum angle between upper arm and trunk: 17.609243309477737
Maximum angle between trunk and knee: 179.49459575663076


Repetition: 2
Minimum angle between upper arm and forearm: 59.16284362394888
Maximum angle between upper arm and trunk: 15.576634910075907
Maximum angle between trunk and knee: 178.19536679530816


Repetition: 3
Minimum angle between upper arm and forearm: 56.966527333681576
Maximum angle between upper arm and trunk: 12.341906800369866
Maximum angle between trunk and knee: 176.17208672616178


Repetition: 4
Minimum angle between upper arm and forearm: 57.17860602993818
Maximum angle between upper arm and trunk: 13.747707988458025
Maximum angle between trunk and knee: 176.0324785100747


Repetition: 5
Minimum angle between upper arm and forearm: 55.63501548784464
Maximum angle between upper arm and trunk: 16.06354080395719
Maximum angle between trunk and knee: 176.4768013563409


Repetition: 

#   Dataset Cleaning

In [6]:
for n in df['Angle_array'].values:
    print(n.size)

99
99
99
18
18
18
54
54
54
75
75
75
76
76
76
77
77
77
66
66
66
69
69
69
74
74
74
66
66
66
68
68
68
70
70
70
70
70
70
65
65
65
112
112
112
88
88
88
71
71
71
24
24
24
11
11
11
29
29
29
6
6
6
52
52
52
68
68
68
67
67
67
71
71
71
24
24
24
11
11
11
29
29
29
6
6
6
52
52
52
68
68
68
65
65
65
38
38
38
41
41
41
4
4
4
40
40
40
51
51
51
52
52
52
28
28
28
15
15
15
61
61
61
54
54
54
55
55
55
61
61
61
38
38
38
43
43
43
48
48
48
47
47
47
51
51
51
55
55
55
52
52
52
50
50
50
43
43
43
44
44
44
50
50
50
54
54
54
51
51
51
51
51
51
47
47
47
49
49
49
48
48
48
48
48
48
48
48
48
51
51
51
50
50
50
47
47
47
48
48
48
89
89
89
97
97
97
101
101
101
92
92
92
12
12
12
94
94
94
114
114
114
86
86
86


In [64]:
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out


In [65]:
from sklearn.model_selection import train_test_split
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn import metrics

y = df['Label']
x = df['Angle_array']


In [66]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
X_train.head()

112    [12.9264378624353, 12.9264378624353, 12.633566...
1      [0.10820696782771888, 0.11090423210451548, 0.2...
184    [16.229971599461322, 16.229971599461322, 16.29...
96     [162.67913717666974, 160.69139644975107, 153.5...
15     [176.63298773474833, 176.63298773474833, 175.5...
Name: Angle_array, dtype: object

In [10]:
m = numpy_fillna(X_train.values)
print(type(m))
for n in m:
    print(n)

<class 'numpy.ndarray'>
[16.117349388782753 16.117349388782753 16.117349388782753
 16.117349388782753 16.117349388782753 15.598927041415198
 15.598927041415198 15.030724417519314 15.030724417519314
 15.030724417519314 15.100588127319076 15.632746515422715
 15.826064587383216 17.993441331058015 18.903945487733324
 18.903945487733324 18.903945487733324 18.903945487733324
 18.889666497488406 18.889666497488406 18.889666497488406
 19.06289432319125 19.389930006639243 19.846417616914234
 19.846417616914234 19.846417616914234 18.612243706965828
 18.489697279377193 17.79485403538462 17.756701564496353 17.37951899536914
 17.13753762152046 16.99820875597564 16.450360978023127 16.175810636175814
 15.176218111103971 13.683178230558902 13.682366243796453
 13.19300769365439 12.747017364017578 12.616173724812468
 12.616173724812468 12.616173724812468 12.616173724812468
 12.616173724812468 12.660846202484754 13.259453414949608
 14.604294106840136 14.605442895354647 14.605442895354647
 14.605442895354

In [67]:
X_train.shape

(180,)

In [68]:
y_train.shape

(180,)

In [69]:
X_test.shape

(45,)

In [70]:
k_range = range(1, 2) #26
scores = {}
scores_list = []
for k in k_range:
    knn_clf = KNeighborsTimeSeriesClassifier(n_neighbors=k, metric="dtw")
    knn_clf.fit(numpy_fillna(X_train.values), y_train)
    predicted_labels = knn_clf.predict(numpy_fillna(X_test.values))
    acc = metrics.accuracy_score(y_test, predicted_labels)
    scores[k] = acc
    print("Correct classification rate:", acc)
    print('\n')
    print(metrics.classification_report(y_test, predicted_labels))
    print(metrics.f1_score(y_test, predicted_labels, average='macro'))


Correct classification rate: 1.0


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        45

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

1.0
